# DFE+SNN core power - Samna only - v0.39.9

1. Power on XA3, do not provide any clocks to XA3 (remember to disable the main clock as well via FPGA settings). The measured power (P0) at VDDHD is the total DFE + SNN Core static power
   
2. Deploy a network and feed spikes via SAER_I interface and run SNN core in parallel, the measured power (P1) at VDDHD is P0 + dynamic SNN core power -> P1 - P0 = dynamic SNN core power. Note:
    * P1 contains minor power from SAER_I processing, it is considered to be very small and can be ignored
    * P1 is measured with SNN core in FA mode (hm ... in fact spikes could be fed via SAER_I in a blind and async way while SNN core is set to real mode, but let's first try FA mode for SNN core)


3. Deploy a network and activate DIG_MIC path as normal, the measured power (P2) at VDDHD is P0 + dynamic SNN core power + dynamic DFE power -> P2 - P1 = dynamic DFE power. Note:
    * P2 is measured with SNN core in real mode

### Initialization and connection to the board

In [1]:
import samna
print(f'Samna version {samna.__version__}')

# Open the device and connect it to the power monitor.
xylo_node = samna.device.open_device("XyloAudio3TestBoard")

power_monitor = xylo_node.get_power_monitor()
sink_pm = samna.graph.sink_from(power_monitor.get_source_node())
stopwatch = xylo_node.get_stop_watch()
io = xylo_node.get_io_module()

xylo_model = xylo_node.get_model()
source = samna.graph.source_to(xylo_model.get_sink_node())

# We are only interested in Readout events, so we make a filter graph to filter only these events.
# Important note: `graph` needs to be kept alive for the filter to work.
graph = samna.graph.EventFilterGraph()
_, etf, readout_sink = graph.sequential([xylo_model.get_source_node(), "XyloAudio3OutputEventTypeFilter", samna.graph.JitSink()])
etf.set_desired_type('xyloAudio3::event::Readout')


## With the network, clock frequency can not go low as 6.25
clock_frequencies = [12.5, 25, 50]
ts = 1e-3


power_measurements_625 = []
power_measurements_125 = []
power_measurements_25 = []
power_measurements_50 = []

Samna version 0.39.9.0


## Create a configuration to map a network on the chip

small == SynNet([24, 24, 24]) 

trained == LIFTorch([63,63,63])

In [2]:
network = "small"

In [3]:
from rockpool.nn.networks import SynNet
from rockpool.devices.xylo.syns65302 import config_from_specification, mapper

xylo_config = []

if network == "trained":
    from rockpool.nn.modules import LIFTorch
    import warnings
    warnings.filterwarnings("ignore")
    
    ckpt = '../../../docs/devices/xylo-a3/model_sample/to_deploy_inXylo.json'
    
    # - Loading trained model architecture parameters
    arch_params = {'n_classes': 1,
    'n_channels': 16,
    'size_hidden_layers':[63, 63, 63],
    'time_constants_per_layer':[3,7,7],
    'tau_syn_base': 0.02,
    'tau_mem': 0.02,
    'tau_syn_out': 0.02,
    'neuron_model': LIFTorch,
    'dt': 0.00994,
    'output': 'vmem'}
    
    # - Instantiating the model backbone and loading trained checkpoint
    syn_model = SynNet(** arch_params)
    syn_model.load(ckpt)
    
    import rockpool.transform.quantize_methods as q
    
    # getting the model specifications using the mapper function
    spec = mapper(syn_model.as_graph(), weight_dtype='float', threshold_dtype='float', dash_dtype='float')
    # quantizing the model
    spec.update(q.channel_quantize(**spec))
    
    xylo_config, is_valid, msg = config_from_specification(**spec)
    
elif network == "small":
    # network to be configured in each step
    xylo_config = samna.xyloAudio3.configuration.XyloConfiguration()

print(xylo_config)

WARNING    /home/vleite/SynSense/rockpool/rockpool/nn/networks/__init__.py:15: UserWarning: This module needs to be ported to the v2 API.
  warnings.warn(f"{err}")
 [py.warnings]
WARNING    /home/vleite/SynSense/rockpool/rockpool/nn/networks/__init__.py:20: UserWarning: This module needs to be ported to the v2 API.
  warnings.warn(f"{err}")
 [py.warnings]


xyloAudio3::configuration::XyloConfiguration(operation_mode=OperationMode.AcceleratedTime, input_source=InputSource.SpikeEvents, synapse2_enable=0, bias_enable=0, time_resolution_wrap=0, output_counter_wrap=0, enable_hibernation_mode=0, hibernation_mode_time_resolution_wrap=0, input=xyloAudio3::configuration::InputConfig(weight_bit_shift=0, weights={ 0 }, syn2_weights={ 0 }), hidden=xyloAudio3::configuration::HiddenConfig(weight_bit_shift=0, weights={ 0 }, syn2_weights={ 0 }, neurons={  }, aliasing=0), readout=xyloAudio3::configuration::ReadoutConfig(weight_bit_shift=0, weights={ 0 }, neurons={  }), analog_frontend=xyloAudio3::configuration::AnalogFrontendConfig(automatic_gain_control=xyloAudio3::configuration::AutomaticGainControlConfig(enable_pga_fixed_gain=1, pga_fixed_gain_index_value=0, rise_average_bit_shift=3, fall_average_bit_shift=16, reliable_max_hysteresis=5, max_sample_number=195312, wait_time_vectors={ 48828 69053 84572 97656 109183 119603 129187 138106 146484 154408 16194

## Create evolve function with pure Samna to activate chip processing

In [4]:
def get_current_timestep():
    """Utility function to obtain the current timestep.

    The `Readout` event always contains the current timestep so we could simply save it.
    But in case you forgot what timestep was last processed, you can obtain it as follows.
    """
    source.write([samna.xyloAudio3.event.TriggerReadout()])
    evts = readout_sink.get_n_events(1, timeout=3000)
    assert(len(evts) == 1)
    return evts[0].timestep


def evolve(input):
    """Continue to evolve the model with the given input.

    Args:
        input (list[list[int]]): Per timestep a list of integers to specify the number of spikes to send to that `neuron_id`.
                                Max number of spikes per timestep for a neuron is 15. All lists must have the same length.

    Returns:
        readouts (list[Readout]): The `Readout` events for the given timesteps.
    """
    timestep_count = len(input)
    if not timestep_count:
        return []

    start_timestep = get_current_timestep() + 1
    final_timestep = start_timestep + timestep_count - 1
    # print("** To be processed timesteps: ", [timestep for timestep in range(start_timestep, final_timestep + 1)])

    input_events_list = []
    for i, spike_counts in enumerate(input):
        timestep = start_timestep + i
        for neuron_id, count in enumerate(spike_counts):
            spikes = [samna.xyloAudio3.event.Spike(neuron_id=neuron_id, timestep=timestep)] * count
            input_events_list.extend(spikes)

    input_events_list.append(samna.xyloAudio3.event.TriggerProcessing(target_timestep = final_timestep + 1))
    source.write(input_events_list)

    events = readout_sink.get_n_events(timestep_count, timeout=3000)
    assert(len(events) == timestep_count)

    # for idx, ev in enumerate(events):
    #         print(f"{idx}: {ev}")

    return events

## P0: Static power of DFE + SNN Core

> 1. Feed PDM_CLK externally
> 3. Disable main clock

In [5]:
# Feed PDM_CLK externally
xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 1

# Disable main clock
io.write_config(0x0008, 0)


In [6]:
# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{avgs[0]:.3f} mW \tAFE core:\t{avgs[1]:.3f} mW\tDFE+SNN core:\t{avgs[2]:.3f} mW')


io:	0.009 mW 	AFE core:	0.013 mW	DFE+SNN core:	0.143 mW


## P1: P0 + dynamic SNN core power

2. Deploy a network and feed spikes via SAER_I interface and run SNN core in parallel
    * The measured power (P1) at VDDHD is P0 + dynamic SNN core power -> P1 - P0 = dynamic SNN core power.
    * Note:
        * P1 contains minor power from SAER_I processing, it is considered to be very small and can be ignored
        * P1 is measured with SNN core in FA mode (hm ... in fact spikes could be fed via SAER_I in a blind and async way while SNN core is set to real mode, but let's first try FA mode for SNN core)

### Calling `apply_configuration` for mapping network and activate chip in Accelerated Time mode


In [7]:
power_consumption = []

for clock in clock_frequencies:
    
    config = samna.xyloAudio3.XyloAudio3TestBoardDefaultConfig()
    config.main_clock_frequency = int(clock * 1000000)
    xylo_node.reset_board_soft(config)

    input_count = 3
    hidden_count = input_count
    output_count = input_count
    xylo_config.input.weights = [[127, 0, 0], [0, 127, 0], [0, 0, 127]] # shape(input_count, hidden_count)
    xylo_config.hidden.weights = [[0, 0, 0], [0, 0, 0], [0, 0, 0]] # shape(hidden_count, hidden_count)
    xylo_config.hidden.neurons = [samna.xyloAudio3.configuration.HiddenNeuron(threshold=127, v_mem_decay=1, i_syn_decay=1)] * hidden_count
    xylo_config.readout.weights = [[127, 0, 0], [0, 127, 0], [0, 0, 127]] # shape(hidden_count, output_count)
    xylo_config.readout.neurons = [samna.xyloAudio3.configuration.OutputNeuron(threshold=127, v_mem_decay=1, i_syn_decay=1)] * output_count
    
    # Configure Xylo in Accelerated-Time mode and work with input spike events.
    xylo_config.operation_mode = samna.xyloAudio3.OperationMode.RealTime
    xylo_config.input_source = samna.xyloAudio3.InputSource.SpikeEvents

    # In Accelerated-Time mode, we can use automatic state monitoring by setting the neuron IDs we want to monitor.
    # Output Vmem and spikes are always activated, so we only monitor the hidden neurons.
    xylo_config.debug.monitor_neuron_v_mem = [i for i in range(hidden_count)]
    xylo_config.debug.monitor_neuron_spike = [i for i in range(hidden_count)]
    # Output Isyn is unavailable by default, so we add hidden and output neurons.
    xylo_config.debug.monitor_neuron_i_syn = [i for i in range(hidden_count + output_count)]

    xylo_node.get_model().apply_configuration(xylo_config)
    xylo_node.get_model().close_ram_access()

    # Start the stopwatch to enable time-stamped power sampling.
    stopwatch.start()
    
    # Start sampling power on all channels at a rate of 100 Hz.
    power_monitor.start_auto_power_measurement(100)

    # Start the filter graph
    graph.start()

    # Send spikes to trigger output neuron 0
    readouts = evolve([
        [2, 0, 0],
        [1, 0, 0],
        [0, 0, 0],
        ])
    assert(readouts[-1].output_spikes == [1, 0, 0])
    
    # Send spikes to trigger output neuron 1
    readouts = evolve([
        [0, 2, 0],
        [0, 1, 0],
        [0, 0, 0],
        ])
    assert(readouts[-1].output_spikes == [0, 1, 0])
    
    # Send spikes to trigger output neuron 2
    readouts = evolve([
        [0, 0, 2],
        [0, 0, 1],
        [0, 0, 0],
        ])
    assert(readouts[-1].output_spikes == [0, 0, 1])
    
    assert(readouts[-1].timestep == 8) # Last processed timestep is 8

    # Auto power measurement streams PowerMeasurement events continuously.
    # To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
    events = sink_pm.get_n_events(900, timeout=4000)
    
    # Stop the automatic power measurement process.
    power_monitor.stop_auto_power_measurement()

    # We have finished processing so that we can stop the graph.
    graph.stop()  
    
    # Calculate the average power consumption for each channel over 3 seconds.
    counts = [0, 0, 0]
    sums = [0, 0, 0]
    for e in events:
        assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
        sums[e.channel] += e.value
        counts[e.channel] += 1
    
    # Compute the averages and convert to milliwatts (W -> mW).
    avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]
    power_consumption+= [avgs]

for clock, power in zip(clock_frequencies, power_consumption):
    print(f'Clock:\t{clock} MHz\nio:\t{power[0]:.3f} mW \tAFE core:\t{power[1]:.3f} mW\tDFE+SNN core:\t{power[2]:.3f} mW')

    match clock:
        case 6.25:
            power_measurements_625.append(power)
        case 12.5:
            power_measurements_125.append(power)
        case 25:
            power_measurements_25.append(power)
        case 50:
            power_measurements_50.append(power)


RuntimeError: Event input is not supported in RealTime mode.
Time resolution wrap must be at least 221 for the given network.
Vmem monitoring is only supported in accelerated-time mode.
Isyn monitoring is only supported in accelerated-time mode.
spike monitoring is only supported in accelerated-time mode.


### Calling `apply_configuration` with same small network for mapping network and digital microphone configuration as in `XyloMonitor` with `RealTime` mode, no chip activation (i.e., not sending `TriggerProcessing`):


In [8]:
power_consumption = []
for clock in clock_frequencies:

    config = samna.xyloAudio3.XyloAudio3TestBoardDefaultConfig()
    config.main_clock_frequency = int(clock * 1000000)
    xylo_node.reset_board_soft(config)

    main_clk_freq = clock * 1e6  # in Hz
    tr_wrap = int(ts * main_clk_freq)
    
    input_count = 3
    hidden_count = input_count
    output_count = input_count
    xylo_config.input.weights = [[127, 0, 0], [0, 127, 0], [0, 0, 127]] # shape(input_count, hidden_count)
    xylo_config.hidden.weights = [[0, 0, 0], [0, 0, 0], [0, 0, 0]] # shape(hidden_count, hidden_count)
    xylo_config.hidden.neurons = [samna.xyloAudio3.configuration.HiddenNeuron(threshold=127, v_mem_decay=1, i_syn_decay=1)] * hidden_count
    xylo_config.readout.weights = [[127, 0, 0], [0, 127, 0], [0, 0, 127]] # shape(hidden_count, output_count)
    xylo_config.readout.neurons = [samna.xyloAudio3.configuration.OutputNeuron(threshold=127, v_mem_decay=1, i_syn_decay=1)] * output_count
    
    xylo_config.operation_mode = samna.xyloAudio3.OperationMode.RealTime
    xylo_config.digital_frontend.filter_bank.dn_enable = False
    xylo_config.time_resolution_wrap = tr_wrap
    xylo_config.debug.always_update_omp_stat = True
    xylo_config.digital_frontend.filter_bank.use_global_iaf_threshold = False
    xylo_config.input_source = samna.xyloAudio3.InputSource.DigitalMicrophone
    # - the ideal sdm clock ratio depends on the main clock rate
    # -- int(clock / 1.56 / 2 - 1)
    xylo_config.debug.sdm_clock_ratio = 24
    xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 0
    xylo_config.digital_frontend.pdm_preprocessing.clock_edge = 0
    
    # - Disable internal state monitoring
    xylo_config.debug.monitor_neuron_v_mem = []
    xylo_config.debug.monitor_neuron_spike = []
    xylo_config.debug.monitor_neuron_i_syn = []

    xylo_node.get_model().apply_configuration(xylo_config)
    xylo_node.get_model().close_ram_access()


    # Activate the chip processing
    source.write([samna.xyloAudio3.event.TriggerProcessing()])

    # Start the stopwatch to enable time-stamped power sampling.
    stopwatch.start()
    
    # Start sampling power on all channels at a rate of 100 Hz.
    power_monitor.start_auto_power_measurement(100)

    # Start the filter graph
    graph.start()

    # Auto power measurement streams PowerMeasurement events continuously.
    # To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
    events = sink_pm.get_n_events(900, timeout=4000)
    
    # Stop the automatic power measurement process.
    power_monitor.stop_auto_power_measurement()

    # We have finished processing so that we can stop the graph.
    graph.stop()  
    
    # Calculate the average power consumption for each channel over 3 seconds.
    counts = [0, 0, 0]
    sums = [0, 0, 0]
    for e in events:
        assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
        sums[e.channel] += e.value
        counts[e.channel] += 1
    
    # Compute the averages and convert to milliwatts (W -> mW).
    avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]
    power_consumption+= [avgs]

for clock, power in zip(clock_frequencies, power_consumption):
    print(f'Clock:\t{clock} MHz\nio:\t{power[0]:.3f} mW \tAFE core:\t{power[1]:.3f} mW\tDFE+SNN core:\t{power[2]:.3f} mW')
    
    match clock:
        case 6.25:
            power_measurements_625.append(power)
        case 12.5:
            power_measurements_125.append(power)
        case 25:
            power_measurements_25.append(power)
        case 50:
            power_measurements_50.append(power)


Clock:	12.5 MHz
io:	1.463 mW 	AFE core:	0.016 mW	DFE+SNN core:	0.466 mW
Clock:	25 MHz
io:	1.529 mW 	AFE core:	0.015 mW	DFE+SNN core:	0.782 mW
Clock:	50 MHz
io:	1.634 mW 	AFE core:	0.015 mW	DFE+SNN core:	1.425 mW


In [2]:
import samna

def read_register(addr):
    source.write([samna.xyloAudio3.event.ReadRegisterValue(address = addr)])
    events = sink.get_n_events(1, 3000)
    assert(len(events) == 1)
    return events[0].data

ctrl1 = 0x0001
ctrl2 = 0x0002
ctrl3 = 0x0003
tr_wrap = 0x0004
hm_tr_wrap = 0x0005
clk_ctrl = 0x0006
clk_div = 0x0007
pwr_ctrl1 = 0x0008
pwr_ctrl2 = 0x0009
pwr_ctrl3 = 0x000A
pwr_ctrl4 = 0x000B
pad_ctrl = 0x000C
ie1 = 0x000E
ie2 = 0x000F
out_ctrl = 0x0011
monsel = 0x0166
mon_grp_sel = 0x0167
dbg_ctrl1 = 0x0168
dbg_stat1 = 0x0171
dfe_ctrl = 0x001B
ivgen = 0x0015
ivgen2 = 0x0016
ivgen3 = 0x0017
ivgen4 = 0x0018
ivgen5 = 0x0019
ivgen6 = 0x001A
adctest = 0x016E

# create access to xylo input/output
source = samna.graph.source_to(xylo_model.get_sink_node())
sink   = samna.graph.sink_from(xylo_model.get_source_node())

## Accelerated Time pure Samna

In [1]:
import samna

clock = 6.25

# Open the device and connect the source and sink nodes so we can communicate with Xylo.
xylo_node = samna.device.open_device("XyloAudio3TestBoard")
xylo = xylo_node.get_model()
source = samna.graph.source_to(xylo.get_sink_node())

config = samna.xyloAudio3.XyloAudio3TestBoardDefaultConfig()
config.main_clock_frequency = int(clock * 1000000)
config.saer_clock_frequency = int(config.main_clock_frequency / 8)
xylo_node.reset_board_soft(config)

# We are only interested in Readout events, so we make a filter graph to filter only these events.
# Important note: `graph` needs to be kept alive for the filter to work.
graph = samna.graph.EventFilterGraph()
_, etf, readout_sink = graph.sequential([xylo.get_source_node(), "XyloAudio3OutputEventTypeFilter", samna.graph.JitSink()])
etf.set_desired_type('xyloAudio3::event::Readout')


# Create a basic network with a simple one-to-one mapping of input to output neurons.
xylo_config = samna.xyloAudio3.configuration.XyloConfiguration()
input_count = 3
hidden_count = input_count
output_count = input_count
xylo_config.input.weights = [[127, 0, 0], [0, 127, 0], [0, 0, 127]] # shape(input_count, hidden_count)
xylo_config.hidden.weights = [[0, 0, 0], [0, 0, 0], [0, 0, 0]] # shape(hidden_count, hidden_count)
xylo_config.hidden.neurons = [samna.xyloAudio3.configuration.HiddenNeuron(threshold=127, v_mem_decay=1, i_syn_decay=1)] * hidden_count
xylo_config.readout.weights = [[127, 0, 0], [0, 127, 0], [0, 0, 127]] # shape(hidden_count, output_count)
xylo_config.readout.neurons = [samna.xyloAudio3.configuration.OutputNeuron(threshold=127, v_mem_decay=1, i_syn_decay=1)] * output_count

# Configure Xylo in Accelerated-Time mode and to work with input spike events.
xylo_config.operation_mode = samna.xyloAudio3.OperationMode.AcceleratedTime
xylo_config.input_source = samna.xyloAudio3.InputSource.SpikeEvents

# In Accelerated-Time mode we can use automatic state monitoring, by setting the neuron ids we want to monitor.
# Output Vmem and spikes are always activated, so we only monitor the hidden neurons.
xylo_config.debug.monitor_neuron_v_mem = [i for i in range(hidden_count)]
xylo_config.debug.monitor_neuron_spike = [i for i in range(hidden_count)]
# Output Isyn is not available by default, so we add both hidden and output neurons.
xylo_config.debug.monitor_neuron_i_syn = [i for i in range(hidden_count + output_count)]

# Send the configuration to Xylo
xylo.apply_configuration(xylo_config)


def get_current_timestep():
    """Utility function to obtain the current timestep.

    The `Readout` event always contains the current timestep so we could simply save it.
    But in case you forgot what timestep was last processed, you can obtain it as follows.
    """
    source.write([samna.xyloAudio3.event.TriggerReadout()])
    evts = readout_sink.get_n_events(1, timeout=3000)
    assert(len(evts) == 1)
    return evts[0].timestep


def evolve(input):
    """Continue to evolve the model with the given input.

    Args:
        input (list[list[int]]): Per timestep a list of integers to specify the number of spikes to send to that `neuron_id`.
                                Max number of spikes per timestep for a neuron is 15. All lists must have the same length.

    Returns:
        readouts (list[Readout]): The `Readout` events for the given timesteps.
    """
    timestep_count = len(input)
    if not timestep_count:
        return []

    start_timestep = get_current_timestep() + 1
    final_timestep = start_timestep + timestep_count - 1
    print("** To be processed timesteps: ", [timestep for timestep in range(start_timestep, final_timestep + 1)])

    input_events_list = []
    for i, spike_counts in enumerate(input):
        timestep = start_timestep + i
        for neuron_id, count in enumerate(spike_counts):
            spikes = [samna.xyloAudio3.event.Spike(neuron_id=neuron_id, timestep=timestep)] * count
            input_events_list.extend(spikes)

    input_events_list.append(samna.xyloAudio3.event.TriggerProcessing(target_timestep = final_timestep + 1))
    source.write(input_events_list)

    events = readout_sink.get_n_events(timestep_count, timeout=3000)
    assert(len(events) == timestep_count)

    for idx, ev in enumerate(events):
            print(f"{idx}: {ev}")

    return events


# Now we are ready to send spikes to our network and see how the SNN core reacts.
# Start the filter graph, otherwise the events aren't propagated.
graph.start()

# sink.get_events()
# print("power control 1:", hex(read_register(0x2)))
# print("power control 2:", hex(read_register(pwr_ctrl2)))
# print("power control 3:", hex(read_register(pwr_ctrl3)))
# print("power control 4:", hex(read_register(pwr_ctrl4)))

# start_address = 0x0001B540
# length = 50#hidden_count + output_count

# # - Generate read events
# read_events = [
#     samna.xyloAudio3.event.ReadMemoryValue(addr)
#     for addr in range(start_address, start_address + length)
# ]

# source.write(read_events)
# len_events = len(read_events)
# events = sink.get_n_events(len_events, 3000)
# assert(len(events) == len_events)
# print(events)

print(config.saer_clock_frequency)

# Send spikes to trigger output neuron 0
readouts = evolve([
    [2, 0, 0],
    [1, 0, 0],
    [0, 0, 0],
    ])

print(readouts[-1])
assert(readouts[-1].output_spikes == [1, 0, 0])

# Send spikes to trigger output neuron 1
readouts = evolve([
    [0, 2, 0],
    [0, 1, 0],
    [0, 0, 0],
    ])
print(readouts[-1])
assert(readouts[-1].output_spikes == [0, 1, 0])

# Send spikes to trigger output neuron 2
readouts = evolve([
    [0, 0, 2],
    [0, 0, 1],
    [0, 0, 0],
    ])
print(readouts[-1])
assert(readouts[-1].output_spikes == [0, 0, 1])

assert(readouts[-1].timestep == 8) # Last processed timestep is 8

# We have finished processing, so we can stop the graph.
graph.stop()

781250
** To be processed timesteps:  [0, 1, 2]
0: xyloAudio3::event::Readout(timestep=0, neuron_v_mems={ 0 0 0 }, neuron_i_syns={ 127 0 0 0 0 0 }, hidden_spikes={ 1 0 0 }, output_v_mems={ 0 0 0 }, output_spikes={ 0 0 0 })
1: xyloAudio3::event::Readout(timestep=1, neuron_v_mems={ 0 0 0 }, neuron_i_syns={ 127 0 0 64 0 0 }, hidden_spikes={ 1 0 0 }, output_v_mems={ 64 0 0 }, output_spikes={ 0 0 0 })
2: xyloAudio3::event::Readout(timestep=2, neuron_v_mems={ 64 0 0 }, neuron_i_syns={ 64 0 0 96 0 0 }, hidden_spikes={ 0 0 0 }, output_v_mems={ 1 0 0 }, output_spikes={ 1 0 0 })
xyloAudio3::event::Readout(timestep=2, neuron_v_mems={ 64 0 0 }, neuron_i_syns={ 64 0 0 96 0 0 }, hidden_spikes={ 0 0 0 }, output_v_mems={ 1 0 0 }, output_spikes={ 1 0 0 })
** To be processed timesteps:  [3, 4, 5]
0: xyloAudio3::event::Readout(timestep=3, neuron_v_mems={ 64 0 0 }, neuron_i_syns={ 32 127 0 48 0 0 }, hidden_spikes={ 0 1 0 }, output_v_mems={ 48 0 0 }, output_spikes={ 0 0 0 })
1: xyloAudio3::event::Readout(t

True

## Real time - pure samna

In [ ]:
import samna
import time

# Open the device and connect the source and sink nodes so we can communicate with Xylo.
board = samna.device.open_device("XyloAudio3TestBoard")
xylo = board.get_model()
source = samna.graph.source_to(xylo.get_sink_node())
sink = samna.graph.sink_from(xylo.get_source_node())

clock = 12.5

config = samna.xyloAudio3.XyloAudio3TestBoardDefaultConfig()
config.main_clock_frequency = int(clock * 1000000)
board.reset_board_soft(config)

# In Real-Time mode we have to specify a timestep duration.
# Default clock frequency was not changed, so we use that to calculate `tr_wrap`.
dt_s = 0.2 # 200 milliseconds
board_config = samna.xyloAudio3.XyloAudio3TestBoardDefaultConfig()
main_clock_frequency_hz = board_config.main_clock_frequency
tr_wrap = main_clock_frequency_hz * dt_s

# Configure Xylo in Real-Time mode and configure the timestep duration.
xylo_config = samna.xyloAudio3.configuration.XyloConfiguration()
xylo_config.operation_mode = samna.xyloAudio3.OperationMode.RealTime
xylo_config.time_resolution_wrap = int(tr_wrap)

# Define a simple network configuration
input_count = 3
hidden_count = 5
output_count = 2
xylo_config.input.weights = [[1] * hidden_count] * input_count
xylo_config.hidden.weights = [[1] * hidden_count] * hidden_count
xylo_config.hidden.neurons = [samna.xyloAudio3.configuration.HiddenNeuron(threshold=16, v_mem_decay=1, i_syn_decay=1)] * hidden_count
xylo_config.readout.neurons = [samna.xyloAudio3.configuration.OutputNeuron(threshold=16, v_mem_decay=1, i_syn_decay=1)] * output_count
xylo_config.readout.weights = [[1] * output_count] * hidden_count

# Configure the input source.
use_digital_mic = True
if use_digital_mic:
    # Choose PDM as input source.
    xylo_config.input_source = samna.xyloAudio3.InputSource.DigitalMicrophone
    # We need to set `clock_direction` to 1 (Xylo output), because there is no external clock.
    xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 1
    xylo_config.digital_frontend.pdm_preprocessing.clock_edge = 0
    # Xylo clock frequency for PDM sampling can be influenced here.
    xylo_config.debug.sdm_clock_ratio = 24
else:
    # Choose ADC as input source.
    xylo_config.input_source = samna.xyloAudio3.InputSource.AnalogMicrophone
    # Disable automatic gain control by setting a fixed gain.
    xylo_config.analog_frontend.automatic_gain_control.enable_pga_fixed_gain = True
    xylo_config.analog_frontend.automatic_gain_control.pga_fixed_gain_index_value = 0

xylo.apply_configuration(xylo_config)

# Communication with the device is asynchronous, so we don't know when the configuration is finished.
# Normally this is not a problem, because events are guaranteed to be sent in order.
# But for this script we want to let Xylo run for a specific amount of time, so we need to synchronise somehow,
# the easiest way is to read a register and wait for the response.
source.write([samna.xyloAudio3.event.ReadRegisterValue(address=0)])
events = sink.get_n_events(1, timeout=1000)
assert len(events) == 1 and isinstance(events[0], samna.xyloAudio3.event.RegisterValue)

# Now we are ready to start processing, the chip keeps running indefinitely after this command.
source.write([samna.xyloAudio3.event.TriggerProcessing()])

# Normally one would continuously read the events and process them (e.g. plotting).
# But for this example we just read events for 3 seconds.
last_timestep = -1
read_until = time.time() + 3.0
while (now:= time.time()) < read_until:
    remaining_time_ms = (read_until - now) * 1000
    events = sink.get_events_blocking(int(remaining_time_ms))
    if events:
        print("Received: ", events)
        for ev in events:
            if isinstance(ev, samna.xyloAudio3.event.Readout):
                last_timestep = ev.timestep

# With a 200ms timestep, around 15 timesteps should have passed in 3 seconds
# However, this can vary due to communication delays, so can not be relied upon
print(f"Last timestep: {last_timestep}")

print("Continuing in Real-Time mode...")
# Xylo will keep running in Real-Time mode indefinitely
# To stop, you can use `source.write([samna.xyloAudio3.event.TriggerProcessing(target_timestep=0)])`
# or reapply a configuration